In [3]:
import pandas as pd
import numpy as np

In [ ]:
# 노원 선정 학원 정리

In [5]:
gj_all = pd.read_csv("../기존 입지 분석 이용 데이터/광진구 구글맵 학원 좌표.csv", encoding = 'cp949')
gj_all

,Unnamed: 0.1,Unnamed: 0,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계
0,0,0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0
1,1,1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0
2,2,2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0
3,3,3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0
4,4,4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0
...,...,...,...,...,...,...,...,...
782,782,782,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0
783,783,783,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0
784,784,784,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0
785,785,785,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0


In [8]:
gj_all=gj_all.drop(labels = ['Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
gj_all

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0
...,...,...,...,...,...,...
782,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0
783,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0
784,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0
785,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0


In [12]:
# longtitude 경도 latitude 위도

In [10]:
from tqdm import tqdm
from tqdm import trange

In [11]:
from haversine import haversine

In [193]:
<<예시 모아둔 BLOCK>>

# haversine 사용법 usage example   
kyobo = (37.504030, 127.024099) # 교보 문고 위도/경도
gangnam = (37.497175,127.027926) # 강남역 위도/경도
haversine(kyobo, gangnam) * 1000 # km to m

# tuple usage example
location1 = tuple([data.loc[0, '위도'], data.loc[0, '경도']])
location1

# 열 삭제
data.drop(labels = '학원수', axis = 1)

NameError: name 'data' is not defined

In [13]:
data = gj_all.copy()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787 entries, 0 to 786
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   학원명         787 non-null    object 
 1   도로명주소       787 non-null    object 
 2   위도          787 non-null    float64
 3   경도          787 non-null    float64
 4   정원합계        787 non-null    int64  
 5   일시수용능력인원합계  785 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 37.0+ KB


In [15]:
# 주소 없는 학원 제거
data.dropna(subset = ['위도'], inplace = True)

In [16]:
# 둘다 00인 학원 삭제
idx = list(data[data['일시수용능력인원합계']+data['정원합계']<=0].index)
data.drop(idx, axis=0, inplace=True)

In [17]:
# 인원 결측치 삭제

data[data['일시수용능력인원합계'].isna()]
data.dropna(subset = ['일시수용능력인원합계'], inplace = True)

In [18]:
# 일시수용 & 정원합계 비교 후 적은 값으로
data.loc[data['정원합계'] >= data['일시수용능력인원합계'], '인원'] = data['일시수용능력인원합계']
data.loc[data['정원합계'] < data['일시수용능력인원합계'], '인원'] = data['정원합계']

In [19]:
data = data.reset_index(drop=True)

In [20]:
data

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0,24.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0,40.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0,53.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0,90.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0,59.0
...,...,...,...,...,...,...,...
780,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0,77.0
781,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0,68.0
782,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0,48.0
783,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0,9.0


In [21]:
# 해당 학원 기준 반경 500m 학원 수 총합 counting
for i in range(len(data)) :
    cnt = 0
    location1 = tuple([data.loc[i, '위도'], data.loc[i, '경도']])
    for j in range(len(data)):
        if i==j:
            continue
        else:
            location2 = tuple([data.loc[j, '위도'], data.loc[j, '경도']])
            if haversine(location1, location2)*1000<=500 :
                cnt += 1
    data.loc[i, '근처학원수'] = cnt-1

In [22]:
# 해당 학원 기준 반경 500m 학원수강생 총합 counting

for i in range(len(data)) :
    s = 0
    location1 = tuple([data.loc[i, '위도'], data.loc[i, '경도']])
    for j in range(len(data)):
        if i==j:
            continue
        else:
            location2 = tuple([data.loc[j, '위도'], data.loc[j, '경도']])
            if haversine(location1, location2)*1000<=500 :             
                s += data.loc[j, '인원']
    data.loc[i, '근처수강생총합'] = s

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   학원명         785 non-null    object 
 1   도로명주소       785 non-null    object 
 2   위도          785 non-null    float64
 3   경도          785 non-null    float64
 4   정원합계        785 non-null    int64  
 5   일시수용능력인원합계  785 non-null    float64
 6   인원          785 non-null    float64
 7   근처학원수       785 non-null    float64
 8   근처수강생총합     785 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 55.3+ KB


In [24]:
store = pd.read_csv("../기존 입지 분석 이용 데이터/광진구 휴게음식점 좌표.csv", encoding = "cp949")
store

,Unnamed: 0,도로명주소,업태구분명,위도,경도
0,0,서울특별시 광진구 뚝섬로 513 (자양동),커피숍,37.531507,127.080562
1,1,"서울특별시 광진구 광나루로56길 85, 9층동 014호 (구의동,테크노마트)",커피숍,37.534937,127.095709
2,2,"서울특별시 광진구 광나루로56길 85, 2층동 D082호 (구의동,테크노마트)",패스트푸드,37.534937,127.095709
3,3,"서울특별시 광진구 광나루로56길 85, 7층동 D082호 (구의동,테크노마트)",패스트푸드,37.534937,127.095709
4,4,"서울특별시 광진구 광나루로56길 85, A023호 (구의동, 테크노마트 지하1층)",패스트푸드,37.534937,127.095709
...,...,...,...,...,...
1362,1362,"서울특별시 광진구 구의로 70, 1층 (구의동)",커피숍,37.537839,127.089116
1363,1363,"서울특별시 광진구 아차산로39길 38-1, 1층 (자양동)",커피숍,37.538141,127.078882
1364,1364,"서울특별시 광진구 능동로32길 72, 1층 (능동)",커피숍,37.555448,127.078579
1365,1365,"서울특별시 광진구 능동로41길 6, 1층 (중곡동)",커피숍,37.560145,127.079585


In [25]:
# 근처 상점 counting (유동인구)
for i in range(len(data)) :
    s = 0
    location1 = tuple([data.loc[i, '위도'], data.loc[i, '경도']])
    for j in range(len(store)):
        location2 = tuple([store.loc[j, '위도'], store.loc[j, '경도']])
        if haversine(location1, location2)*1000<=500 :
            s += 1
    data.loc[i, '근처휴게음식점수'] = s

In [27]:
data.head()

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원,근처학원수,근처수강생총합,근처휴게음식점수
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0,24.0,221.0,11724.0,188.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0,40.0,46.0,2822.0,73.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0,53.0,47.0,2527.0,83.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0,90.0,42.0,2699.0,45.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0,59.0,44.0,1553.0,62.0


In [29]:
# 근처 횡단보도 counting
crosswalk = pd.read_csv("../기존 입지 분석 이용 데이터/광진구 횡단보도 좌표.csv", encoding = 'cp949')
crosswalk

,노드링크 유형,시군구명,읍면동명,경도,위도
0,NODE,광진구,군자동,127.076049,37.556554
1,NODE,광진구,군자동,127.074172,37.547463
2,LINK,광진구,광장동,127.100091,37.540782
3,NODE,광진구,군자동,127.079171,37.556957
4,NODE,광진구,군자동,127.078326,37.556825
...,...,...,...,...,...
1006,NODE,광진구,군자동,127.078553,37.557313
1007,NODE,광진구,군자동,127.078677,37.557267
1008,LINK,광진구,자양동,127.075553,37.533153
1009,NODE,광진구,군자동,127.077068,37.557878


In [30]:
for i in range(len(data)) :
    s = 0
    location1 = tuple([data.loc[i, '위도'], data.loc[i, '경도']])
    for j in range(len(crosswalk)):
        location2 = tuple([crosswalk.loc[j, '위도'], crosswalk.loc[j, '경도']])
        if haversine(location1, location2)*1000<=500 :
            s += 1
    data.loc[i, '근처횡단보도수'] = s

In [31]:
data

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원,근처학원수,근처수강생총합,근처휴게음식점수,근처횡단보도수
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0,24.0,221.0,11724.0,188.0,124.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0,40.0,46.0,2822.0,73.0,79.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0,53.0,47.0,2527.0,83.0,53.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0,90.0,42.0,2699.0,45.0,77.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0,59.0,44.0,1553.0,62.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
780,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0,77.0,55.0,2363.0,77.0,61.0
781,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0,68.0,49.0,2082.0,35.0,59.0
782,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0,48.0,212.0,11728.0,196.0,131.0
783,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0,9.0,23.0,1758.0,132.0,53.0


In [32]:
# 근처 사고 counting
accident = pd.read_csv("../기존 입지 분석 이용 데이터/광진구 2012~2021 사망 교통사고 데이터 좌표.csv", encoding = "cp949")
accident

,발생년,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,경도,위도
0,2012,주간,수,1,1,0,0,0,127.086638,37.537284
1,2012,야간,수,1,1,0,0,0,127.071670,37.547921
2,2012,주간,토,1,1,0,0,0,127.094631,37.534842
3,2012,야간,금,1,1,0,0,0,127.110621,37.548560
4,2012,주간,월,1,1,0,0,0,127.073653,37.559601
...,...,...,...,...,...,...,...,...,...,...
93,2020,야,금,1,0,0,0,0,127.078491,37.560798
94,2021,주,토,1,1,0,1,0,127.066558,37.531434
95,2021,야,목,1,0,0,0,0,127.083550,37.536643
96,2021,야,화,1,0,0,0,0,127.072723,37.557794


In [33]:
# 근처 사고 counting
for i in range(len(data)) :
    s = 0
    location1 = tuple([data.loc[i, '위도'], data.loc[i, '경도']])
    for j in range(len(accident)):
        location2 = tuple([accident.loc[j, '위도'], accident.loc[j, '경도']])
        if haversine(location1, location2)*1000<=500 :
            s = s + sum(accident.loc[j, ['사망자수','부상자수','중상자수','경상자수','부상신고자수']])
    data.loc[i, '사고건수'] = s

In [34]:
data

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원,근처학원수,근처수강생총합,근처휴게음식점수,근처횡단보도수,사고건수
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0,24.0,221.0,11724.0,188.0,124.0,4.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0,40.0,46.0,2822.0,73.0,79.0,12.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0,53.0,47.0,2527.0,83.0,53.0,17.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0,90.0,42.0,2699.0,45.0,77.0,14.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0,59.0,44.0,1553.0,62.0,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
780,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0,77.0,55.0,2363.0,77.0,61.0,10.0
781,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0,68.0,49.0,2082.0,35.0,59.0,12.0
782,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0,48.0,212.0,11728.0,196.0,131.0,6.0
783,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0,9.0,23.0,1758.0,132.0,53.0,17.0


In [35]:
gj_all = data.copy()

In [37]:
gj_all.to_csv('광진구데이터프레임.csv',index=False)

In [39]:
# 데이터프레임 저장
gj = pd.read_csv("광진구데이터프레임.csv")
gj

,학원명,도로명주소,위도,경도,정원합계,일시수용능력인원합계,인원,근처학원수,근처수강생총합,근처휴게음식점수,근처횡단보도수,사고건수
0,생명을살리는수학창의교육보습학원,서울특별시 광진구 구의강변로 106,37.539900,127.094819,24,85.0,24.0,221.0,11724.0,188.0,124.0,4.0
1,시엘음악학원,서울특별시 광진구 용마산로21길 27,37.563485,127.085939,40,63.0,40.0,46.0,2822.0,73.0,79.0,12.0
2,성신에듀학원,서울특별시 광진구 자양로22길 24,37.539490,127.084692,255,53.0,53.0,47.0,2527.0,83.0,53.0,17.0
3,청람보습학원,서울특별시 광진구 용마산로 100,37.562608,127.087319,90,90.0,90.0,42.0,2699.0,45.0,77.0,14.0
4,상명보습학원,서울특별시 광진구 자양로 181-1,37.543898,127.084563,59,115.0,59.0,44.0,1553.0,62.0,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
780,하이클래스보습학원,서울특별시 광진구 뚝섬로 655,37.531600,127.082320,136,77.0,77.0,55.0,2363.0,77.0,61.0,10.0
781,해법영어최선엘리트학원,서울특별시 광진구 천호대로 136길 46,37.547763,127.090071,170,68.0,68.0,49.0,2082.0,35.0,59.0,12.0
782,에이스에듀케이션학원,서울특별시 광진구 아차산로 471,37.540413,127.093944,48,128.0,48.0,212.0,11728.0,196.0,131.0,6.0
783,숨피아노교습소,서울특별시 광진구 동일로26길 46,37.544708,127.068316,40,9.0,9.0,23.0,1758.0,132.0,53.0,17.0
